# Cleaning Oscar Data

The oscar database was found on kaggle's website in the form of a csv file. The data in found in this file was used to create two dictionaries. A director dictionary storing the years a director recieved a Oscar nominations and the years a director won. The second dictionary stored a list of oscar nominated films based on the year it was nominated and the number of nominations. Given that some films have the same title as others. The dictionary of films that have been nominated stored the years that film was nominated. Some manual edits were required for some director names. Particular names with multiple spelling and/or special characters.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## oscar_database.csv found on Kaggle datasets
oscarDF = pd.read_csv('data/kraggle_data/oscar_database.csv')
oscarDF.tail(8)

,Year,Ceremony,Award,Winner,Name,Film
9961,2015,88,Jean Hersholt Humanitarian Award,1.0,NaN,Debbie Reynolds
9962,2015,88,Honorary Award,1.0,NaN,Spike Lee
9963,2015,88,Honorary Award,1.0,NaN,Gena Rowlands
9964,2016,89,Directing,1.0,Damien Chazelle,La La Land
9965,2016,89,Directing,NaN,Mel Gibson,Hacksaw Ridge
9966,2016,89,Directing,NaN,Barry Jenkins,Moonlight
9967,2016,89,Directing,NaN,Kenneth Lonergan,Manchester by the Sea
9968,2016,89,Directing,NaN,Denis Villeneuve,Arrival


Given that the oscars occur in Feburary. It was decided to go with latter date. Therefore 1927/1928 will map to 1928.

In [2]:
year = oscarDF.Year.unique()
year_int = range(1928, 2017)
year_dict = dict(zip(year, year_int))

oscarDF['Year'] = oscarDF.Year.map(year_dict)

# Oscar Nominated Directors

The subset of oscar data for nominated directors contained 439 rows. Some cleaning was required due to entries in the Name and Film columns being swapped for 50 entries. Also the dataset contained up to the awards from 2015 films, the directors nominated for 2016 films were added to the set. 15 directors names contained multiple spelling. For example Alejandro G Inarritu vs Alejandro Gonzales Inarritu. These directors name spelling were standardized to make indexing on them easier.

In [3]:
## Getting Oscar Nominated Directors
Oscar_DirectorDF = oscarDF[oscarDF.Award == 'Directing']
Oscar_DirectorDF.Winner = (Oscar_DirectorDF.Winner == 1.0)*1


Oscar_DirectorDF['film'] = Oscar_DirectorDF.Film.str.strip()
Oscar_DirectorDF['director'] = Oscar_DirectorDF.Name.str.strip()
Oscar_DirectorDF.drop(['Film','Name'], axis=1,inplace=True)

## Set index
Oscar_DirectorDF.index = range(len(Oscar_DirectorDF))

Oscar_DirectorDF.shape

(439, 6)

In [4]:
## Director dictionary with list of years nominated and list of years won
director_dict = {}

for d in Oscar_DirectorDF.director.unique():
    tempDF = Oscar_DirectorDF[Oscar_DirectorDF['director']==d]
    years_nominated = tempDF.Year.values
    tempDF = tempDF[tempDF['Winner']==1]
    years_won = tempDF.Year.values
    director_dict[d] = {'oscar_noms':years_nominated,'oscar_wins':years_won}
    
len(director_dict)

225

## Main Movie Data
The main dataframe comes from a script used to retrieve movie data from Box Office Mojo webpage. The data was stored as a pickle file 

In [5]:
# storing the data from the picklefile to a dataframe
import pickle
from datetime import datetime

filename = 'data/bom_data/all-movie-data.pkl'
picklefile = open(filename, 'rb')
all_movies_data = pickle.load(picklefile, encoding='bytes')

mainDF = pd.DataFrame(all_movies_data)
mainDF.columns

Index(['1-title', '2-release_date', '3-closing_date', 'budget', 'director',
       'distributor', 'dom_total_gross', 'domestic_total_adj_gross', 'genre',
       'intl_total_gross', 'oscar_noms', 'oscar_wins', 'producers', 'rating',
       'runtime_mins', 'theaters', 'url'],
      dtype='object')

In [6]:
# Renaming columns
mainDF.rename(columns = {'1-title': 'title', 
                         '2-release_date': 'release_date', 
                         '3-closing_date': 'closing_date'}, inplace = True)
mainDF.shape

(17748, 17)

In [7]:
# Dropping 'url' column as it's not required
mainDF.drop('url', inplace=True, axis=1)

# Grouping a vast list of subgenres into a few main genres
by_genre = mainDF.groupby('genre')

In [8]:
# Esxtracting a movie's release season from it's release date
def get_season(date):
    '''Return season given month as int.'''
    months = (range(1, 13))
    seasons = ['winter', 'winter', 'spring', 'spring', 
           'summer', 'summer', 'summer', 'summer', 
           'fall', 'fall', 'holiday', 'holiday']
    seasons = dict(zip(months, seasons))
    try:
        return seasons[date.month]
    except:
        return ''
    
# map raw genres to simple genres
def get_simple_genre(g):
    '''Recategorize raw genre into simple genre.'''
    raw_genres = tuple(by_genre.count().index)
    simple_genres = ['Act_Adven', 'Act_Adven', 'Act_Adven', 'Act_Adven', 'Act_Adven', 'Act_Adven', 
                 'Act_Adven', 'Act_Adven', 'Act_Adven', 'Act_Adven', 
                 'Animation', 'Comedy', 'Comedy', 'Comedy', 'Other', 
                 'Crime_Thriller', 'Crime_Thriller', 'Crime_Thriller', 'Crime_Thriller', 
                 'Documentary', 'Drama', 'Drama', 'Family', 'Family', 'Family', 'Fantasy_Scifi', 'Fantasy_Scifi', 
                 'Fantasy_Scifi', 'Foreign', 'Foreign', 'Foreign', 'Historical', 'Historical', 
                 'Horror', 'Horror', 'Horror', 'Other', 'Musical', 'Musical', 'None', 
                 'Historical', 'Historical', 'Historical', 'Historical', 'Historical', 
                 'Romance', 'Romance', 'Romance', 'Romance', 'Fantasy_Scifi', 'Fantasy_Scifi', 
                 'Fantasy_Scifi', 'Fantasy_Scifi', 'Fantasy_Scifi', 'Fantasy_Scifi', 
                 'Fantasy_Scifi', 'Sports', 'Sports', 'Sports', 'Sports',  'Crime_Thriller', 
                 'None', 'War', 'War', 'War', 'Western', 'Western']
    genres = dict(zip(raw_genres, simple_genres))
    try:
        return genres[g]
    except:
        return ''

# clean up ratings
# Replacing 'Unknown', 'Unrated', 'None', 'Not Yet Rated' by empty strings
def get_clean_rating(r):
    rm_ratings = ['Unknown', 'Unrated', 'None', 'Not Yet Rated']
    return '' if r in rm_ratings else r

def recat(df, func, new_col, old_col):
    '''Recategorize variable in new column based on data in old col
    and provided function. 
    Args: 
    df -- data frame
    func -- function to apply
    new_col (str) -- name of new column (to store result of function)
    old_col (str) -- name of old column (on which to apply function)
    '''
    df[new_col] = df.loc[:, old_col].apply(func) 
    
    
recat(mainDF, get_season, 'season', 'release_date')
recat(mainDF, get_clean_rating, 'clean_rating', 'rating')
recat(mainDF, get_simple_genre, 'new_genre', 'genre')

Using the release and close date the number of days a film was in theaters was added as a new feature.

In [9]:
# calculating the number of days a movie ran in theaters
start_date = mainDF.release_date
end_date = mainDF.closing_date
days_in_theater = (end_date-start_date).dt.days

mainDF['days_in_theater'] = days_in_theater

In [10]:
# Extracting and adding year of release as a new feature
year = [t.year for t in pd.to_datetime(start_date)]
mainDF['year'] = year

# One Hot Encoding Ratings and Season:

In [11]:

for column in mainDF['season'].unique():
    mainDF[column] =mainDF['season'].apply(lambda x: 1 if x == column else 0)

for column in mainDF['clean_rating'].unique():
    mainDF[column] = mainDF['clean_rating'].apply(lambda x: 1 if x == column else 0)
    
mainDF.head()

,title,release_date,closing_date,budget,director,distributor,dom_total_gross,domestic_total_adj_gross,genre,intl_total_gross,...,,PG-13,R,PG,G,GP,M/PG,NC-17,M,X
0,The A-Team,2010-06-11,2010-09-16,110000000.0,[Joe Carnahan],Fox,77222099.0,86741300.0,Action,177238796.0,...,0,1,0,0,0,0,0,0,0,0
1,A.C.O.D.,2013-10-04,2013-11-07,NaN,,The Film Arcade,175705.0,187900.0,Comedy,NaN,...,0,1,0,0,0,0,0,0,0,0
2,A.I. Artificial Intelligence,2001-06-29,NaT,100000000.0,[Steven Spielberg],Warner Bros.,78616689.0,124036600.0,Sci-Fi,235926552.0,...,0,1,0,0,0,0,0,0,0,0
3,A Aa,2016-06-01,2016-06-16,NaN,,Blue Sky Cinema,2313043.0,2366000.0,Comedy / Drama,NaN,...,1,0,0,0,0,0,0,0,0,0
4,Aaja Nachle,2007-11-30,2007-12-20,NaN,,Yash Raj,484108.0,628400.0,Foreign,6773493.0,...,1,0,0,0,0,0,0,0,0,0


In [12]:
mainDF.columns

Index(['title', 'release_date', 'closing_date', 'budget', 'director',
       'distributor', 'dom_total_gross', 'domestic_total_adj_gross', 'genre',
       'intl_total_gross', 'oscar_noms', 'oscar_wins', 'producers', 'rating',
       'runtime_mins', 'theaters', 'season', 'clean_rating', 'new_genre',
       'days_in_theater', 'year', 'summer', 'fall', 'holiday', 'winter',
       'spring', '', 'PG-13', 'R', 'PG', 'G', 'GP', 'M/PG', 'NC-17', 'M', 'X'],
      dtype='object')

In [13]:
# Dropping unnecessary columns 
drop_columns =['closing_date','intl_total_gross','X', 'NC-17', 'M','','season']
## The Dataframe contains movies with mature rating, which will not be included in the final dataframe. 
mainDF = mainDF.drop(drop_columns, axis=1)

Films with  Mature rating were dropped from the dataset. Given that these films are never considered for the oscars. 

In [14]:
rating_drop = ['Unknown', 'Unrated', 'None', 'Not Yet Rated']
mainDF = mainDF[mainDF['rating']!=rating_drop[0]]
mainDF = mainDF[mainDF['rating']!=rating_drop[1]]
mainDF = mainDF[mainDF['rating']!=rating_drop[2]]
mainDF = mainDF[mainDF['rating']!=rating_drop[3]]

In [15]:
mainDF.shape

(11206, 29)

# Common Directors from both dataframes

The Following code was used to help add missing oscar directors to the main dataframe. A new csv file was created to perform some manual changes. There ws no unique identification for each director between the two data sets. The list of directors in the main dataframe was queried for matches each oscar nominated director. Similar to cleaning the oscar nominated directors, there were many inconsistanceys in the spelling of a name. This required manual changes to the dataset. 

In [16]:
# ## Set of director from main dataframe
# cleanedDF = pd.read_csv('manual_cleaned_added director.csv',encoding='utf-8')
# movie_director = cleanedDF.director.astype(str).tolist()
# string_remove = ["'","[","]",'"']

# def remove_char(L, remove):
#     for i in range(len(L)):
#         s = L[i]
#         for ch in remove:
#             if ch in s:
#                 s = s.replace(ch,"")
#         L[i] = s
#     return L

# movie_director = set(remove_char(movie_director,string_remove))
# movie_director.remove('')

In [17]:
# ## Set of common directors between the two dataframes
# oscar_director = set(Oscar_DirectorDF.director)
# common_director = set.intersection(oscar_director, movie_director) 
# len(common_director)

In [18]:
# missing_directors = set.difference(oscar_director,movie_director) ## set of the 84 missing directors

# possible_matches = {}

# for md in missing_directors:
    
#     for d in movie_director:
        
#         a = md.split(" ")
#         b = d.split(" ")
#         inter = list(set(b).intersection(a))
#         if len(inter)>0:

#             if md in possible_matches.keys():
#                 possible_matches[md].append(d)
#             else:
#                 possible_matches[md] = [d]
        
# len(possible_matches)

# ##mainDF.to_csv('manual_clean2.csv', encoding='utf-8' )

In [19]:
# loading the manually cleaned director's data into a datframe
mainDF2 = pd.read_csv("data/final_clean_data/manual_cleaned_added director.csv",encoding='utf-8') 
mainDF2.shape

(11331, 29)

# Filling in missing  Oscar Nomination to movies

In [20]:
# Finding the number of movies having an oscar nominations
len(mainDF2[mainDF2['oscar_noms']>0])

1263

In [21]:
nominated_films = set(oscarDF.Film)
nominated_films.update(oscarDF.Name)
nominated_films = nominated_films.intersection(mainDF2.title)
nominated_films.remove(np.nan)

len(nominated_films)

638

In [22]:
# Making a dict of films which contains information like number of nominations and the number of 
# nominations it recieved till that year
# dict format example:
# nom_dict["titanic"] ={'year': 1991, "noms": 3}
nom_dict = {}
for f in nominated_films:
    
    noms = len(oscarDF[oscarDF['Film']==f].Year.values)
    year = oscarDF[oscarDF['Film']==f].Year.values
    if noms == 0:
        noms = len(oscarDF[oscarDF['Name']==f].Year.values)
        year = oscarDF[oscarDF['Name']==f].Year.values
    
    nom_dict[f] ={'year':year[0],'noms': noms}

In [23]:
'''
Using the nom_dict to update the number of oscar nominations a movie recieved by querying the 
dict by film title and release year
'''
for t in nominated_films:
    if t in nom_dict.keys():
        
        df = mainDF2[mainDF2['title']==t]
        oscar_info = nom_dict[t]
        
        for i in range(len(df)):
            
            row = df.iloc[i]
            year =int(row.year)
            n = row.oscar_noms
            
            if year == oscar_info['year']:
            
                if n ==0:

                    mainDF2['oscar_noms'][df.index[i]]= oscar_info['noms']
                    

In [24]:
len(mainDF2[mainDF2['oscar_noms']>0])

1354

# Adding director reputation 

In this section we will add two columns 'director_prev_nom' and 'director_prev_win' to the dataframe.We'll than later use this two columns to analyse how a director's previous oscar nominations and wins affect a movie's chance of getting nominated.
1. Oscar_DirectorDF - contains the all the directors that have been nominated or won an oscar atleast once.
2. filmDirectors - contains all the directors of the films that are in our main dataframe.

We'll query the Oscar_DirectorDF by the director's name and movie year to get a count of the nominations and wins a director has recieved upto the movie's release year and update this info in the main dataframe.

After all the processing, we are left with 197 directors which have been nominated or won an oscar atleast once.

In [25]:
mainDF2['director_prev_nom'] = pd.Series([0 for x in range(len(mainDF2.index))])
mainDF2['director_prev_win'] = pd.Series([0 for x in range(len(mainDF2.index))])

In [26]:
oscarDirectors = set(Oscar_DirectorDF.director) 
filmDirectors = mainDF2.director.fillna('0')

In [27]:
string_remove = ["'","[","]",'"']
def remove_char(L, remove):
    for i in L.index:
        s = L[i]
        for ch in remove:
            if ch in s:
                s = s.replace(ch,"")
        L[i] = s
    return L

# Removing junk characters from director's names for better matching
directors = remove_char(filmDirectors, string_remove)

In [28]:
#finding the directors that have been nominated oe won an oscar out of all the directors
film_directors = set(directors)
common_directors = set.intersection(film_directors, oscarDirectors)
len(common_directors)

197

In [29]:
## using the director_dict to fill up the previous director nominations and wins for a movie
mainDF2['director'] = directors

for director in common_directors:
    
    oscar_infor = director_dict[director]
    temp = mainDF2[mainDF2['director'] == director]
    
    for i in range(len(temp.index)):
        
        row = temp.iloc[i]
        mainDF2['director_prev_nom'][temp.index[i]]= len([j for j in oscar_infor['oscar_noms'] if j < row.year])
        mainDF2['director_prev_win'][temp.index[i]]= len([j for j in oscar_infor['oscar_wins'] if j < row.year])
        
mainDF2.head()

,title,release_date,budget,director,distributor,dom_total_gross,domestic_total_adj_gross,genre,oscar_noms,oscar_wins,...,winter,spring,PG-13,R,PG,G,GP,M/PG,director_prev_nom,director_prev_win
0,The A-Team,6/11/2010,110000000.0,Joe Carnahan,Fox,77222099.0,86741300.0,Action,0,0.0,...,0,0,1,0,0,0,0,0,0,0
1,A.C.O.D.,10/4/2013,NaN,0,The Film Arcade,175705.0,187900.0,Comedy,0,0.0,...,0,0,1,0,0,0,0,0,0,0
2,A.I. Artificial Intelligence,6/29/2001,100000000.0,Steven Spielberg,Warner Bros.,78616689.0,124036600.0,Sci-Fi,2,0.0,...,0,0,1,0,0,0,0,0,5,2
3,Abandon,10/18/2002,25000000.0,Stephen Gaghan,Paramount,10727683.0,16488500.0,Thriller,0,0.0,...,0,0,1,0,0,0,0,0,0,0
4,The Abandoned,2/23/2007,NaN,0,After Dark Films,1331137.0,1727800.0,Horror,0,0.0,...,1,0,0,1,0,0,0,0,0,0


In [30]:
# Exporting the main dataframe to a csv file
mainDF2.to_csv('test_final.csv',encoding='utf-8',index=False)

# Adding other awards

In this section, we'll add other major nominations secured by a movie like Golden Globe, Guild and Bafta. Films.csv contains all the films and Nominations.csv contains all the nominations grabbed for all movies.We'll also be fetching rotten tomatoes audience and critic score for our movies. We'll make a list of films called common_films that have awards and rotten tomatoes information by doing an intersection on the two datasets.
Finally  we'lluse data from this common list of films to fill in the missing attributes like other awards info, rotten tomateoes scores in our cleaned movies dataframe.
In the end we are left with 1144 movies which have all the awards and rating information

In [31]:
#Read datasets
films = pd.read_csv('data/movie_data/films.csv')

nominations = pd.read_csv('data/movie_data/nominations.csv')


In [32]:
# keep only those which have rt_audience_score
films = films[films['rt_audience_score']>0.0]
len(films)

1767

In [33]:
#keep only those which have rt_critic_score
films = films[films['rt_critic_score']>0.0]
len(films)

1635

In [34]:
#getting the unique number of films in nominations.csv
len(nominations.film.unique())

1816

In [35]:
# Getting the films which are common to both nominations and film dataset
films_set = set(films['film'])
nominations_set = set(nominations['film'])
common_films = nominations_set.intersection(films_set)

In [36]:
# Getting the total number of awards
awards = nominations.award.unique()
awards

array(['Oscar', 'BAFTA', 'Golden Globe', 'Guild'], dtype=object)

In [37]:
#getting common films dataframe 
common_films_df = films[films['film'].isin(common_films)]

In [38]:
# Adding other awards- Oscar, BAFTA, Golden Globe and Guild columns to common films dataframe
import numpy as np
common_films_df.columns
for award in awards:
    common_films_df[award] = 0

In [39]:
# Getting the number of other awards nominations - BAFTA,Oscar, Guild and Golden Globe 
for film in common_films:
    matched_films = nominations[nominations['film']==film]
    for i in range(0,len(matched_films)):
        match = matched_films.iloc[i]
        value = common_films_df.loc[common_films_df['film'] == film, match['award'] ] + 1
        common_films_df.loc[common_films_df['film'] == film, match['award'] ]= value
common_films_df.head()

,bom_domestic,bom_foreign,bom_worldwide,box_office,budget,country,film,imdb_score,metacritic_score,mpaa,...,rt_audience_score,rt_critic_score,running_time,stars_count,writers_count,year,Oscar,BAFTA,Golden Globe,Guild
0,NaN,NaN,NaN,5.2 million,3000000,United States,Elmer Gantry,7.9,NaN,APPROVED,...,86.0,96.0,146 minutes,4.0,1.0,1960,4,3,5,0
1,NaN,NaN,NaN,"1,500,000",805000,United Kingdom,Sons and Lovers,7.3,NaN,NaN,...,54.0,75.0,103 minutes,4.0,1.0,1960,5,1,5,0
2,NaN,NaN,NaN,24.6 million,3000000,United States,The Apartment,8.3,NaN,APPROVED,...,94.0,93.0,125 minutes,3.0,2.0,1960,6,3,4,0
3,NaN,NaN,NaN,NaN,NaN,United Kingdom,The Entertainer,7.2,NaN,NaN,...,73.0,89.0,"107 min., 37 sec.",5.0,1.0,1960,1,1,0,0
4,NaN,NaN,NaN,"2,000,000",2000000,United States,Inherit the Wind,NaN,NaN,NaN,...,91.0,91.0,128 minutes,6.0,1.0,1960,2,2,2,0


In [40]:
# checking how many films have nominated for oscars out of common films
oscar_df = common_films_df[common_films_df['Oscar']>0]
len(oscar_df)

963

In [41]:
# loading the cleaned dataset of films
cleaned_films = mainDF2
cleaned_films.head()

,title,release_date,budget,director,distributor,dom_total_gross,domestic_total_adj_gross,genre,oscar_noms,oscar_wins,...,winter,spring,PG-13,R,PG,G,GP,M/PG,director_prev_nom,director_prev_win
0,The A-Team,6/11/2010,110000000.0,Joe Carnahan,Fox,77222099.0,86741300.0,Action,0,0.0,...,0,0,1,0,0,0,0,0,0,0
1,A.C.O.D.,10/4/2013,NaN,0,The Film Arcade,175705.0,187900.0,Comedy,0,0.0,...,0,0,1,0,0,0,0,0,0,0
2,A.I. Artificial Intelligence,6/29/2001,100000000.0,Steven Spielberg,Warner Bros.,78616689.0,124036600.0,Sci-Fi,2,0.0,...,0,0,1,0,0,0,0,0,5,2
3,Abandon,10/18/2002,25000000.0,Stephen Gaghan,Paramount,10727683.0,16488500.0,Thriller,0,0.0,...,0,0,1,0,0,0,0,0,0,0
4,The Abandoned,2/23/2007,NaN,0,After Dark Films,1331137.0,1727800.0,Horror,0,0.0,...,1,0,0,1,0,0,0,0,0,0


In [42]:
# Adding other awards- Oscar, BAFTA, Golden Globe and Guild columns to cleaned films dataframe and 
# setting the default count to 0
for award in awards:
    cleaned_films[award] = 0

In [43]:
# Fetching films common to cleaned movies dataframe and common_films_df dataframe
clean_common_films = common_films_df[common_films_df['film'].isin(cleaned_films['title'])]

In [44]:
len(clean_common_films)

1144

In [45]:
# Making a list of  films that are a part of cleaned_movies dataframe
# Keys: film_names
# Values: awards, rt_audience_score, rt_critic_score
# Sample format:
# cleaned_films_dict = { "titanic":{'BAFTA': 1, 'Guild:2', 'Golden Globe:3','rt_audience_score: 65.0,
#                                  'rt_critic_score': 85.9}}

cleaned_films_dict={}
for i in range(0,len(clean_common_films)):
    film = clean_common_films.iloc[i]
    film_name = film['film']
    cleaned_films_dict[film_name]={}
    for award in awards:
        cleaned_films_dict[film_name][award]= film[award]
    cleaned_films_dict[film_name]['rt_audience_score'] = film['rt_audience_score']
    cleaned_films_dict[film_name]['rt_critic_score'] = film['rt_critic_score']

        
len(cleaned_films_dict)

1144

In [46]:
# Filling up missing other awards count, rotten tomatoes critic score , rotten tomatoes audience score
# to the cleaned movies dataframe
for key, value in cleaned_films_dict.items():
    for award in awards:
        cleaned_films.loc[cleaned_films['title']==key,award]=value[award]
    cleaned_films.loc[cleaned_films['title']==key, 'rt_critic_score'] = value['rt_critic_score']
    cleaned_films.loc[cleaned_films['title']==key, 'rt_audience_score'] = value['rt_audience_score'
                                                                       ]

In [47]:
# Fetching the number of films nominted for eaach categrory of awards
for award in awards:
    print("{}:{}".format(award,len(cleaned_films[cleaned_films[award]>0])))

Oscar:799
BAFTA:584
Golden Globe:1076
Guild:362


In [48]:
oscar_df = cleaned_films[cleaned_films['Oscar']>0]
len(oscar_df)

799

In [49]:
# Getting the movies which have mismatched oscar_noms and Oscar fields
mismatched_films=[]
for i in range(0,len(cleaned_films)):
    film = cleaned_films.iloc[i]
#     print(film['title'])
#     print(film['Oscar'])
#     print(film['oscar_noms'])
#     print("__________________")
    if film['oscar_noms']!= film['Oscar']:
            mismatched_films.append(film['title'])
        

len(mismatched_films)       

1164

In [50]:
mismatched_films[:5]


['A.I. Artificial Intelligence',
 'The Abyss',
 'The Accidental Tourist',
 'Across the Universe',
 'Adaptation.']

In [51]:
# Keeping the max of oscar_noms and Oscar fields
# since both specify the number of oscar nominations and 
# it makes sense to keep only one field and drop the redundant column later
test_cleaned_films = cleaned_films.copy()
for i in range(0,len(test_cleaned_films)):
    film = test_cleaned_films.iloc[i]
    if film['oscar_noms']!= film['Oscar']:
        noms = np.maximum(film['oscar_noms'],film['Oscar'])
        test_cleaned_films.loc[test_cleaned_films['title']==film['title'],'oscar_noms'] = noms
        test_cleaned_films.loc[test_cleaned_films['title']==film['title'],'Oscar'] = noms
                    

In [52]:
# Check if there is any mismatch still between oscar_noms and Oscar fields
mismatched_films=[]
for i in range(0,len(test_cleaned_films)):
    film = test_cleaned_films.iloc[i]
    if film['oscar_noms']!= film['Oscar']:
            mismatched_films.append(film['title'])
        

len(mismatched_films)    

0

In [53]:
len(test_cleaned_films[test_cleaned_films['oscar_noms']>0])

1574

In [54]:
len(cleaned_films[cleaned_films['oscar_noms']>0])

1354

In [55]:
# drop the redundant column - 'Oscar' since we already have 'oscar_noms'
test_cleaned_films.drop('Oscar',axis=1,inplace=True)

In [56]:
# Replacing NaNs by 0 in following fields since we cannot use average to determine the relative values
# 1.rt_audience_score
# 2. rt_critic_score
# 3. budget
# 4. domestic_total_adj_gross
# 5. dom_total_gross
test_cleaned_films.rt_audience_score.fillna(0.0, inplace=True)
test_cleaned_films.rt_critic_score.fillna(0.0, inplace=True)
test_cleaned_films.budget.fillna(0.0, inplace=True)
test_cleaned_films.dom_total_gross.fillna(0.0, inplace=True)
test_cleaned_films.domestic_total_adj_gross.fillna(0.0, inplace=True)



In [57]:
test_cleaned_films.columns

Index(['title', 'release_date', 'budget', 'director', 'distributor',
       'dom_total_gross', 'domestic_total_adj_gross', 'genre', 'oscar_noms',
       'oscar_wins', 'producers', 'rating', 'runtime_mins', 'theaters',
       'clean_rating', 'new_genre', 'days_in_theater', 'year', 'summer',
       'fall', 'holiday', 'winter', 'spring', 'PG-13', 'R', 'PG', 'G', 'GP',
       'M/PG', 'director_prev_nom', 'director_prev_win', 'BAFTA',
       'Golden Globe', 'Guild', 'rt_critic_score', 'rt_audience_score'],
      dtype='object')

In [58]:
# Creating a Final Data frame
finalDF = test_cleaned_films

In [59]:
# Dropping films with Genre ='Foriegn' , considering only films produced in USA 
# since we are not accounting for foriegn gross 
# First level filtering on Genre
finalDF = finalDF[finalDF['genre']!= 'Foreign']
finalDF.shape

(10816, 36)

In [60]:
# Dropping films with New Genre ='Foriegn' , considering only films produced in USA 
# since we are not accounting for foriegn gross 
# Second level filtering on Genre

finalDF = finalDF[finalDF['new_genre']!= 'Foreign']
finalDF.shape

(10749, 36)

In [61]:
## Encode Genre, Producers, Directors since they are strings and represent categorial data
dir_dict = {d:i for i, d in enumerate(list(finalDF.director.unique()))}
genre_dict = {g: i for i, g in enumerate(list(finalDF.new_genre.unique()))}
producre_dict = {p: i for i, p in enumerate(list(finalDF.producers.unique()))}

encoded_director = finalDF.director.map(dir_dict)
encoded_genre = finalDF.new_genre.map(genre_dict)
encoded_prod = finalDF.producers.map(producre_dict)

finalDF['encoded_director'] = encoded_director
finalDF['encoded_genre'] = encoded_genre
finalDF['encoded_producer'] = encoded_prod

In [62]:
# ## Normalize Encoded Data (Genre, Producers, Directors) and target variable

# encoded_data =['encoded_director','encoded_genre','encoded_producer']
# for e in encoded_data:
#     finalDF[e]=((finalDF[e]-finalDF[e].min())/(finalDF[e].max()-finalDF[e].min()))
    
# # ## Normalize Target Variable
# # mainDF2['oscar_noms']=((mainDF2['oscar_noms']-mainDF2['oscar_noms'].min())/(mainDF2['oscar_noms'].max()-mainDF2['oscar_noms'].min()))

In [63]:
#Filling NaNs in 'runtime_mins' column by mean of all the entries since most of the films have
#have a runtime that falls into a particular range
average_runtime = finalDF.runtime_mins.mean()
finalDF.runtime_mins.fillna(average_runtime, inplace=True)

# Filling NaNs in 'dom_total_gross' and  'domestic_total_adj_gross' columns by 0
finalDF.budget.fillna(0,inplace=True)
finalDF.dom_total_gross.fillna(0,inplace=True)
finalDF.domestic_total_adj_gross.fillna(0,inplace=True)

In [64]:
#Save Final Cleaned data to train models 
file_name ="data/final_clean_data/final_clean_data.csv"
finalDF.to_csv(file_name, encoding='utf-8', index= False)